# How to use this code?

1. Make a copy of the code. Make sure you save it in the same folder
2. Save your clients file in the incoming folder
3. Edit the code where necessary. See comment lines before the code to check where you have to change the code

# Choose file to process

In [1]:
# Please enter the name of the file after executing this cell (leave empty for test data)
filename = input('Filename of file in /incoming/ folder: ')
client = input('Client\'s name: ')
client = client if client else 'Test'

Filename of file in /incoming/ folder: brondata-dierenapotheek-v3.csv
Client's name: Dierenapotheek


# Import needed libraries

In [2]:
from datetime import datetime
from pathlib import Path
import helpers  # custom helper functions
import numpy as np
import pandas as pd
import pandas.api.types as ptypes

assert pd.__version__ > '0.24.9'   # Need Pandas version 0.25.0 or higher

# Import file and filter data

In [3]:
# Building the full path to the file
path = Path().absolute().parent  # each '.parent' goes one level up - vary as required
if filename:
    file = f'{path}/incoming/{filename}'
else:
    file = f'{path}/samples/sample-dataset.csv'

# Path for outgoing files
path_outgoing = f'{path}/outgoing/'

print('Processing: ', file)
print('For client: ', client)

Processing:  /Users/chiwang/Downloads/python-master/incoming/brondata-dierenapotheek-v3.csv
For client:  Dierenapotheek


In [4]:
# File import
# To do: choose the right columns from the dataset
# for excel: data = pd.read_excel(file) 
# df = pd.DataFrame(data, columns = [''])

df = pd.read_csv(file, sep = None, delimiter = None, usecols = ['email','Orderdate','OrderID','TurnoverLead', 'BuyinLead', 'LeadPhase','LeadStatus'], engine = 'python') 
df.head()

,email,Orderdate,OrderID,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,mirandaverhoeff@ziggo.nl,43465.0,247197691,14.45,NaN,NaN,completed
1,j.hydra1@chello.nl,43465.0,247188831,110.85,NaN,NaN,completed
2,arrowsflight123@gmail.com,43465.0,247150721,116.85,NaN,NaN,completed
3,m.keesmekers@gmail.com,43465.0,247147717,6.40,NaN,NaN,completed
4,scrabble1@kpnmail.nl,43465.0,247145195,51.20,NaN,NaN,completed


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9948 entries, 0 to 9947
Data columns (total 7 columns):
email           9804 non-null object
Orderdate       8924 non-null float64
OrderID         9948 non-null int64
TurnoverLead    9948 non-null float64
BuyinLead       0 non-null float64
LeadPhase       0 non-null float64
LeadStatus      9948 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 544.2+ KB


In [6]:
# Delete empty rows'NaT'
# Check email contains @ -> aanpassen als je geen gebruik maakt van email als client id
# Delete duplicates in orderID -> naam aanmpassen naar naam van de kolom 

df = df.dropna(axis=0, how='any')
df = df[df['email'].str.contains('@')]
df = df.drop_duplicates('OrderID', keep='first')
df.head()

,email,Orderdate,OrderID,TurnoverLead,BuyinLead,LeadPhase,LeadStatus


In [11]:
def floatHourToTime(fh):
    h, r = divmod(fh, 1)
    m, r = divmod(r*60, 1)
    return (
        int(h),
        int(m),
        int(r*60),
    )

orderDate = "Orderdate"
dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(orderDate) - 2)
hour, minute, second = floatHourToTime(orderDate % 1)
dt = dt.replace(hour=hour, minute=minute, second=second)

ValueError: invalid literal for int() with base 10: 'Orderdate'

In [7]:
# Find out the date format with the helper function
date = df['Orderdate'].loc[df['Orderdate'].first_valid_index()]  # first occurence
date_format = helpers.date_format(date)

# Orderdate to datetime. Vul juiste date format in
df['Orderdate'] = pd.to_datetime(df['Orderdate'], format=date_format)

KeyError: None

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Filtering the data (these rules are specific for Maxilia -> delete if not necessary or change to needed values)
# Leadphase = order
# Leadstatus = gefactureerd
df = df[df['LeadPhase'].str.contains('Order')]
df = df[df['LeadStatus'].str.contains('Gefactureerd')]
df.head()

In [ ]:
df.info()

In [ ]:
# Check if the data equals certain conditions
assert ptypes.is_datetime64_any_dtype(df['Orderdate'])
assert df['email'].str.contains('@').all()
assert df['LeadPhase'].eq('Order').all()
assert df['LeadStatus'].eq('Gefactureerd').all()

# Group data per customer

In [ ]:
# Here the data is being grouped by email. 
# To do: change the variables into the right column names of the data set

# The first and last transaction are taken to calculate how many days someone is a customer
# The number of orders (how many OrderID's per customer) per customer is calculated
# The sum of all spend is calculated

df_clv = df.groupby('email').agg(
    first_transaction=('Orderdate', min),
    last_transaction=('Orderdate', max),
    total_transactions=('OrderID', len), 
    total_spent=('TurnoverLead', sum)
) 
df_clv['days_customer'] = (df_clv['last_transaction'] - df_clv['first_transaction']).dt.days
df_clv.drop(['first_transaction', 'last_transaction'], axis=1, inplace=True)
df_clv.head()

In [ ]:
df_clv.columns = ['days_customer', 'total_transactions', 'total_spent']
df_clv.head()

In [ ]:
df_clv.describe()

In [ ]:
# Export to file
# df_clv.to_excel(f'{path_outgoing}{client} - Overzicht_per_klant.xlsx')
df_clv.to_csv(f'{path_outgoing}{client} - Overzicht_per_klant.csv')

# Calculating the total CLV

In [ ]:
# Initiate a dictionary that holds all calculated values
calculated = {}

In [ ]:
# Calculate the average order value

avg_order_value = sum(df_clv['total_spent']) / sum(df_clv['total_transactions'])
calculated['Gemiddelde orderwaarde'] = avg_order_value
print(avg_order_value)

In [ ]:
# Calculate how often someone buys from the company

purchase_frequency = sum(df_clv['total_transactions']) / df_clv.shape[0]
calculated['Frequentie aankoop'] = purchase_frequency
print(purchase_frequency)

In [ ]:
# Calculate how often someone returns
# Output * 100 is the rate in %

repeat_rate = df_clv[df_clv.total_transactions > 1].shape[0] / df_clv.shape[0] 
calculated['Repeat rate'] = repeat_rate
print(repeat_rate)

In [ ]:
# Calculate the churnrate: what % doesn't return after 1 order

churn_rate = 1 - repeat_rate
calculated['Churn rate'] = churn_rate
print(churn_rate)

In [ ]:
# Calculating the average margin of the dataset
# To do: choose the right columns from the dataset

profit = sum(df['TurnoverLead']) - sum(df['BuyinLead'])
calculated['Profit'] = profit
margin = profit / sum(df['TurnoverLead'])
calculated['Margin'] = margin
absolut_margin = profit * margin
calculated['Absolut margin'] = absolut_margin
    
print(profit)    
print(margin)
print(absolut_margin)

In [ ]:
# Calculating the total CLV and the CLV with margin
# If you cannot calculate the margin (as is done in the above cell) you can change margin into the margin number that the company provides you with

total_clv = (avg_order_value * purchase_frequency) / churn_rate 
calculated['CLV'] = total_clv
total_clv_margin = total_clv * margin
calculated['CLV marge'] = total_clv_margin

print(total_clv) # het eerste getal is de totale clv
print(total_clv_margin) # het tweede getal is de clv waarbij rekening is gehouden met de berekende marge

In [ ]:
# In this cell the above numbers are set into a dataset (which is necessary to use them in datastudio)
df_clv_values = pd.DataFrame([calculated], index=['Waarde'])
df_clv_values.head()

In [ ]:
# Export to CSV file
df_clv_values.to_csv(f'{path_outgoing}{client} - CLV_waardes.csv')
# Export to Excel-file use:
# df_clv_values.to_excel(f'{path_outgoing}{client} - CLV_waardes.xlsx')

   # Churn rate per month

In [ ]:
# Grouping data per month to calculate the churn rate per month
df_monthly = df.pivot_table(
    index=['email'],
    columns=pd.Grouper(key='Orderdate', freq='M'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

df_monthly.head()

In [ ]:
repeat_rate = df_monthly[df_monthly > 1].count() / df_monthly[df_monthly > 0].count()
df_churn = repeat_rate.to_frame(name='repeat_rate')

In [ ]:
df_churn['churn_rate'] = 1 - df_churn['repeat_rate']
df_churn.head()


# Churn per 6 months and per year

In [ ]:
# Grouping data per 6 months to calculate the churn rate per 6 months
df_6_months = df.pivot_table(
    index=['email'],
    columns=pd.Grouper(key='Orderdate', freq='6M'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

df_6_months.head()

In [ ]:
repeat_rate = df_6_months[df_6_months > 1].count() / df_6_months[df_6_months > 0].count()
df_churn_6_months = repeat_rate.to_frame(name='repeat_rate_6_months')

In [ ]:
df_churn_6_months['churn_rate_6_months'] = 1 - df_churn_6_months['repeat_rate_6_months']
df_churn_6_months

In [ ]:
# Grouping the data per year to calculate the churn rate per year
df_year = df.pivot_table(
    index=['email'],
    columns=pd.Grouper(key='Orderdate', freq='Y'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

df_year.head()

In [ ]:
repeat_rate = df_year[df_year > 1].count() / df_year[df_year > 0].count()
df_churn_year = repeat_rate.to_frame(name='repeat_rate_year')

In [ ]:
df_churn_year['churn_rate_year'] = 1 - df_churn_year['repeat_rate_year']
df_churn_year

# Transactions, spend and unique customers per month

In [ ]:
# Grouping total transactions, turnover and unique clients per month

df_transacties = df.groupby(pd.Grouper(key='Orderdate', freq='M')).agg(
    {
        'OrderID': len, 
        'TurnoverLead': sum,
        'email': pd.Series.nunique
    }
)

df_transacties.head()

In [ ]:
df_transacties.info()

In [ ]:
# Renaming the columns
df_transacties.columns = ['total_transactions', 'total_spent', 'total_unique_customers']
df_transacties.head(5)

In [ ]:
df_transacties.describe()

In [ ]:
# Export to file
# df_transacties.to_excel(f'{path_outgoing}{client} - Overzicht_per_maand.xlsx')
df_transacties.to_csv(f'{path_outgoing}{client} - Overzicht_per_maand.csv')

# Calculate CLV per month with churn per month

In [ ]:
# Calculating the CLV per month
df_transacties['purchase_frequency_month'] = df_transacties['total_transactions'] / df_transacties['total_unique_customers']
df_transacties['avg_order_value_month'] = df_transacties['total_spent'] / df_transacties['total_transactions'] 
df_transacties['CLV'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_churn['churn_rate']
df_transacties['CLV_margin'] = df_transacties['CLV'] * margin
df_transacties.head()

In [ ]:
# New dataset to calculate the aggregated average CLV
# Drop empty columns
# Add index column to calculate the aggregated average CLV
df_clv_cum= df_transacties[['CLV', 'CLV_margin']].copy()
df_clv_cum.dropna(inplace=True)
df_clv_cum['Index'] = np.arange(1, len(df_clv_cum) + 1)
df_clv_cum['CLV_AVG'] = df_clv_cum['CLV'].cumsum().div(df_clv_cum['Index'])
df_clv_cum['CLV_margin_AVG'] = df_clv_cum['CLV_margin'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

# Calculate CLV per month with churn per 6 months

In [ ]:
# Code used to find the churn that connects to the right months
def find_churn_rate_6_months(date):
    """
    Iterate through the 'df_churn_6_months' dataframe.
    When the date is smaller than the date on the index, 
    return the corresponding churn rate 
    Eg: July < Oct, so return October's churn rate
    """
    for m in df_churn_6_months['churn_rate_6_months'].iteritems():
        if date <= m[0]:
            return m[1]
        
def find_churn_rate_1_year(date):
    """
    Iterate through the 'df_churn_year' dataframe.
    When the date is smaller than the year on the index, 
    return the corresponding churn rate 
    """
    for y in df_churn_year['churn_rate_year'].iteritems():
        if date <= y[0]:
            return y[1]

In [ ]:
# Calculate CLV with Churn per 6 months
# First line of code is used to find the right churn for the corresponding months
df_transacties['churn_rate_6_months'] = df_transacties.index.to_series().apply(lambda x: find_churn_rate_6_months(x))
df_transacties['CLV_churn_6M'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_transacties['churn_rate_6_months']
df_transacties['CLV_margin_churn_6M'] = df_transacties['CLV_churn_6M'] * margin
df_transacties.head(10)

# Calculate CLV per month with churn per year

In [ ]:
# Calculate CLV with churn per year
df_transacties['churn_rate_1_year'] = df_transacties.index.to_series().apply(lambda x: find_churn_rate_1_year(x))
df_transacties['CLV_churn_Y'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_transacties['churn_rate_1_year']
df_transacties['CLV_margin_churn_Y'] = df_transacties['CLV_churn_Y'] * margin
df_transacties.head(10)

# Aggregated average CLV

In [ ]:
# Calculate aggregated average CLV with churn of 6 months and add to dataset
df_clv_cum_6M = df_transacties[['CLV_churn_6M', 'CLV_margin_churn_6M']].copy()
df_clv_cum_6M.dropna(inplace=True)
df_clv_cum['CLV_AVG_6M'] = df_clv_cum_6M['CLV_churn_6M'].cumsum().div(df_clv_cum['Index'])
df_clv_cum['CLV_margin_AVG_6M'] = df_clv_cum_6M['CLV_margin_churn_6M'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

In [ ]:
# Calculate aggregated average CLV with churn of a year and add to dataset
df_clv_cum_Y = df_transacties[['CLV_churn_Y', 'CLV_margin_churn_Y']].copy()
df_clv_cum_Y.dropna(inplace=True)
df_clv_cum['CLV_AVG_Y'] = df_clv_cum_Y['CLV_churn_Y'].cumsum().div(df_clv_cum['Index'])
df_clv_cum['CLV_margin_AVG_Y'] = df_clv_cum_Y['CLV_margin_churn_Y'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

In [ ]:
# Keep dataset with all the values
# CLV_AVG is churn calculated per month, 6M churn is churn over 6 months, Y is churn calculated over a year
df_clv_avg = df_clv_cum[['CLV_AVG', 'CLV_margin_AVG', 'CLV_AVG_6M', 'CLV_margin_AVG_6M', 'CLV_AVG_Y', 'CLV_margin_AVG_Y']].copy()
df_clv_avg.head(10)

In [ ]:
# Export to file
# df_clv_avg.to_excel(f'{path_outgoing}{client} - Gemiddeldes CLV per maand.xlsx')
df_clv_avg.to_csv(f'{path_outgoing}{client} - Gemiddeldes CLV per maand.csv')